### Load data

In [1]:
import numpy as np
X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")
person_train_valid = np.load("person_train_valid.npy")
X_train_valid = np.load("X_train_valid.npy")
y_train_valid = np.load("y_train_valid.npy")
person_test = np.load("person_test.npy")

y_train_valid -= 769
y_test -= 769


print(np.unique(y_train_valid))
print(np.unique(y_test))

[0 1 2 3]
[0 1 2 3]


### Shape of data

In [2]:
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


### Prepare Training Data

In [3]:
import torch
from torch.utils import data

X_train_valid = torch.from_numpy(X_train_valid).float()
y_train_valid = torch.from_numpy(y_train_valid).long()
train_data = data.TensorDataset(X_train_valid, y_train_valid)

person_train_valid = torch.from_numpy(person_train_valid)
person_test = torch.from_numpy(person_test)

In [4]:
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

batch_size = 128
valid_size = 0.2

"""
# convert data to a normalized torch.FloatTensor
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
"""

# obtain training indices that will be used for validation
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders (combine dataset and sampler)
train_loader = data.DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)
valid_loader = data.DataLoader(train_data, batch_size=batch_size, sampler=valid_sampler)

## CNN Architecture

### AlexNet (modified)

In [5]:
import torch

train_on_gpu = torch.cuda.is_available()

# check if CUDA is available
device = torch.device("cuda:0" if train_on_gpu else "cpu")
print(device)
if not train_on_gpu:
    print('Training on CPU ...')
else:
    print('Training on GPU ...')

cuda:0
Training on GPU ...


In [7]:
import torch.nn as nn
import torch.nn.functional as F

class AlexNet(nn.Module):

    def __init__(self, num_classes=4):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv1d(22, 64, kernel_size=22, stride=2),
            # want to capture relation in both dims, pad only time dim
            nn.BatchNorm1d(64, affine=False),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool1d(kernel_size=12, stride=2),
            nn.Dropout(p=0.6),
            nn.Conv1d(64, 192, kernel_size=12),
            nn.BatchNorm1d(192, affine=False),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool1d(kernel_size=3, stride=2),
            nn.Dropout(p=0.6),
            nn.Conv1d(192, 384, kernel_size=4, stride=2),
            nn.BatchNorm1d(384, affine=False),
            nn.LeakyReLU(inplace=True),
            nn.Dropout(p=0.6),
            nn.Conv1d(384, 256, kernel_size=4, stride=2),
            nn.BatchNorm1d(256, affine=False),
            nn.LeakyReLU(inplace=True),
            nn.Dropout(p=0.6),
            nn.Conv1d(256, 256, kernel_size=4),
            nn.BatchNorm1d(256, affine=False),
            nn.LeakyReLU(inplace=True),
            nn.MaxPool1d(kernel_size=4, stride=1),
            
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.BatchNorm1d(256 * 21),
            nn.LeakyReLU(inplace=True),
            nn.BatchNorm1d(256 * 21),
            nn.LeakyReLU(inplace=True),
            nn.Linear(256 * 21, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# create a complete CNN
model = AlexNet()

# move tensors to GPU if CUDA is available
if train_on_gpu:
    model.cuda()

### Train the Model

In [11]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), weight_decay=0.0005, lr=1e-3)

In [13]:
# number of epochs to train the model
n_epochs = 300

valid_loss_min = np.Inf # track change in validation loss

for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    model.train()
    for data, target in train_loader:
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        #print(target.data)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)
        
    ######################    
    # validate the model #
    ######################
    model.eval()
    for data, target in valid_loader:
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        output = model(data)
        loss = criterion(output, target)
        valid_loss += loss.item()*data.size(0)
    
    # calculate average losses
    train_loss = train_loss/len(train_loader.sampler)
    valid_loss = valid_loss/len(valid_loader.sampler)
        
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model_cifar.pt')
        valid_loss_min = valid_loss

Epoch: 1 	Training Loss: 0.533812 	Validation Loss: 0.892710
Validation loss decreased (1.027166 --> 0.892710).  Saving model ...
Epoch: 2 	Training Loss: 0.486990 	Validation Loss: 0.930557
Epoch: 3 	Training Loss: 0.420447 	Validation Loss: 0.945232
Epoch: 4 	Training Loss: 0.467292 	Validation Loss: 0.921304
Epoch: 5 	Training Loss: 0.467665 	Validation Loss: 0.947924
Epoch: 6 	Training Loss: 0.437161 	Validation Loss: 0.932468
Epoch: 7 	Training Loss: 0.391329 	Validation Loss: 0.924639
Epoch: 8 	Training Loss: 0.396134 	Validation Loss: 0.957915
Epoch: 9 	Training Loss: 0.428465 	Validation Loss: 0.983101
Epoch: 10 	Training Loss: 0.424010 	Validation Loss: 0.936174
Epoch: 11 	Training Loss: 0.405984 	Validation Loss: 0.961260
Epoch: 12 	Training Loss: 0.372738 	Validation Loss: 0.972351
Epoch: 13 	Training Loss: 0.352774 	Validation Loss: 0.951336
Epoch: 14 	Training Loss: 0.354456 	Validation Loss: 0.961739
Epoch: 15 	Training Loss: 0.371419 	Validation Loss: 0.948038
Epoch: 16 

### Load the Model with the Lowest Validation Loss

In [14]:
model.load_state_dict(torch.load('model_cifar.pt'))

<All keys matched successfully>

### Test the Trained Network

In [9]:
from torch.utils import data

batch_size = 200

X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).long()
test_data = data.TensorDataset(X_test, y_test)

test_loader = data.DataLoader(test_data, batch_size=batch_size)

In [15]:
# specify the target classes
classes = [769, 770, 771, 772]

# track test loss
test_loss = 0.0
class_correct = list(0. for i in range(4))
class_total = list(0. for i in range(4))

model.eval()
for data, target in test_loader:
    if train_on_gpu:
        data, target = data.cuda(), target.cuda()
    output = model(data)
    loss = criterion(output, target)
    # update test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)
    #print(pred)
    # compare predictions to true label
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    # calculate test accuracy for each object class
    for i in range(correct.shape[0]):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1
# average test loss
test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(4):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Loss: 1.106453

Test Accuracy of   769: 65% (73/111)
Test Accuracy of   770: 41% (53/127)
Test Accuracy of   771: 31% (30/96)
Test Accuracy of   772: 85% (93/109)

Test Accuracy (Overall): 56% (249/443)
